# Cruise Control Simulation

### Libraries Used

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import control as ct

## Process Model

### Vehicle Dynamics

In [3]:
def vehicle_update(t, x, u, params={}):
    """"
    Vehicle Dynamics

    Parameters:
    x (array) : System state, car velocity in m/s
    u (array) : System input, [throttle, gear, road_slope], 
            where throttle is a float between 0 and 1, gear is an 
            integer between 1 and 5, and road_slope is in rad.
    Returns:
    dv (float) : Vehicle acceleration

    """
    # Get system parameters
    m = params.get('m', 1000.)              # vehicle mass, kg
    g = params.get('g', 9.8)                # gravitational constant, m/s^2
    alpha = params.get(
        'alpha', [40, 25, 16, 12, 10])      # gear ratio / wheel radius

    # Define variables for vehicle state and inputs
    v = x[0]                           # vehicle velocity
    throttle = np.clip(u[0], 0, 1)     # vehicle throttle
    gear = u[1]                        # vehicle gear
    theta = u[2]                       # road slope

    # Force generated by the engine
    omega = alpha[int(gear)-1] * v      # engine angular speed
    F = alpha[int(gear)-1] * motor_torque(omega, params) * throttle

    # Disturbance force 
    # Letting the grade of the road be theta, gravity gives the
    # force Fg = m*g*sin(theta)
    Fg = m * g * sin(theta)

    # Final acceleration on the car
    Fd = Fg
    dv = (F - Fd) / m

    return dv

### Engine Model

In [4]:
def motor_torque(omega, params={}):
    # Set up the system parameters
    Tm = params.get('Tm', 190.)             # engine torque constant
    omega_m = params.get('omega_m', 420.)   # peak engine angular speed
    beta = params.get('beta', 0.4)          # peak engine rolloff

    return np.clip(Tm * (1 - beta * (omega/omega_m - 1)**2), 0, None)